In [ ]:
!pip install tulipy
!pip install pycoingecko
!pip install pandas_ta
!pip install python-binance
!pip install ta


# Gerekli kütüphanlerin importu

In [46]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import tulipy as ti
import csv
import builtins
import matplotlib.pyplot as plt
import yfinance as yf
import json
import webbrowser
from datetime import datetime, timedelta
from binance import Client
from jinja2 import Template
import subprocess
import ta
from ta.momentum import RSIIndicator
from ta.trend import ADXIndicator
from ta.trend import SMAIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands

# Binanceden veri çekme

In [47]:
#Verileri getirme
API_Key = "QkGSZFlPppl7a5xc9qyUs8AGBr0gFLDc2XsWlGacTL73SoHf4UHZbk7gTZLNuj9p"
Secret_Key = "gQmP0g7cnzYniHNZjZeIyX5uerTs13wBi16eGyjViFvntDgzjU7LqVOtKs7y7x1P"
client = Client(API_Key,Secret_Key,tld='us')


# Binance verilerin mumlarını alma

In [48]:
def verileriGetir(sembol, periyot, baslangic, bitis):
  mumlar = client.get_historical_klines(sembol, periyot, baslangic, bitis)
  return mumlar

# Veritabanı oluşturma

In [49]:
#Csv dosyasına kaydetme
def csvOlustur(sembol, mumlar):
  csvDosya = builtins.open(sembol + ".csv", "w", newline='')
  yazici = csv.writer(csvDosya, delimiter=',')
  for mumVerileri in mumlar:
    yazici.writerow(mumVerileri)
  csvDosya.close()

# Girilen sembollerin verisini çekme

In [50]:
sembolList = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "DOGEUSDT","ATOMUSDT"]
def veriCekmeVeCsvOlusturma(sembolList):
    for coin in sembolList:
        csvOlustur(coin, verileriGetir(coin, Client.KLINE_INTERVAL_1DAY, "19 January 2023", "19 April 2024"))
        print(coin, "csv dosyası oluşturuldu")
veriCekmeVeCsvOlusturma(sembolList)


BTCUSDT csv dosyası oluşturuldu
ETHUSDT csv dosyası oluşturuldu
BNBUSDT csv dosyası oluşturuldu
DOGEUSDT csv dosyası oluşturuldu
ATOMUSDT csv dosyası oluşturuldu


# Verilerin Başlığını yazma

In [51]:
okunacakCsv = "BTCUSDT.csv"
basliklar = ['Open_time','Open','High','Low','Close','Volume','Close_time','Quote_asset_volume','Number_of_trades','Taker_buy_base_asset_volume','Taker_buy_quote_asset_volume','Ignore']
df = pd.read_csv(okunacakCsv,names=basliklar)
acilis=df['Open']
print(acilis)

0      20677.13
1      21080.05
2      22675.01
3      22783.64
4      22710.87
         ...   
452    65826.97
453    63380.58
454    63806.00
455    61350.59
456    63447.69
Name: Open, Length: 457, dtype: float64


#TimeStamp Verisini Okunur Hâle Getirme

In [52]:

def zamanHesapla(timestamp):
    return datetime.fromtimestamp(timestamp.timestamp())

def zamanYazdır():
    for zaman in acilis_zamani:
        print(zamanHesapla(zaman))

zamanYazdır()

2024-03-29 20:00:00
2024-03-29 21:00:00
2024-03-29 22:00:00
2024-03-29 23:00:00
2024-03-30 00:00:00
2024-03-30 01:00:00
2024-03-30 02:00:00
2024-03-30 03:00:00
2024-03-30 04:00:00
2024-03-30 05:00:00
2024-03-30 06:00:00
2024-03-30 07:00:00
2024-03-30 08:00:00
2024-03-30 09:00:00
2024-03-30 10:00:00
2024-03-30 11:00:00
2024-03-30 12:00:00
2024-03-30 13:00:00
2024-03-30 14:00:00
2024-03-30 15:00:00
2024-03-30 16:00:00
2024-03-30 17:00:00
2024-03-30 18:00:00
2024-03-30 19:00:00
2024-03-30 20:00:00
2024-03-30 21:00:00
2024-03-30 22:00:00
2024-03-30 23:00:00
2024-03-31 00:00:00
2024-03-31 01:00:00
2024-03-31 02:00:00
2024-03-31 03:00:00
2024-03-31 04:00:00
2024-03-31 05:00:00
2024-03-31 06:00:00
2024-03-31 07:00:00
2024-03-31 08:00:00
2024-03-31 09:00:00
2024-03-31 10:00:00
2024-03-31 11:00:00
2024-03-31 12:00:00
2024-03-31 13:00:00
2024-03-31 14:00:00
2024-03-31 15:00:00
2024-03-31 16:00:00
2024-03-31 17:00:00
2024-03-31 18:00:00
2024-03-31 19:00:00
2024-03-31 20:00:00
2024-03-31 21:00:00


#Rsi hesaplama ve değişken tanımlama

In [53]:

open = df['Open']
close = df['Close']
acilis_zamani = df['Open_time']
high = df['High']
low = df['Low']

periyod = 14 #default rsı değeri
rsi_indicator = RSIIndicator(close=close, window=periyod)
rsi = rsi_indicator.rsi()

print(rsi)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
452    42.055725
453    43.032891
454    38.629413
455    43.862566
456    44.749538
Name: rsi, Length: 457, dtype: float64


#Hareketli Ortlamalar (sma, adx)


In [54]:
sma_indicator = SMAIndicator(close, window=50)
sma_50 = sma_indicator.sma_indicator()
print(sma_50)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
452    67007.7032
453    67195.0892
454    67279.5334
455    67300.5316
456    67354.8946
Name: sma_50, Length: 457, dtype: float64


In [55]:
adx_indicator = ADXIndicator(high, low, close, periyot=14)
adx = adx_indicator.adx()

print(adx)

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
         ...    
452    17.825873
453    18.594031
454    19.786650
455    20.894082
456    21.167109
Name: adx, Length: 457, dtype: float64


#Macd Hesaplama

In [56]:
macd_indicator = MACD(close, window_slow=26, window_fast=12, window_sign=9)
macd = macd_indicator.macd()

print(macd)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
452   -133.090377
453   -387.271583
454   -787.023678
455   -917.819152
456   -979.515786
Name: MACD_12_26, Length: 457, dtype: float64


#Boolinger Bandi hesaplama


In [57]:
bb_indicator = BollingerBands(close, window=20, window_dev=2)
bb = bb_indicator.bollinger_mavg()

print(bb)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
452    68425.7555
453    68146.8350
454    67670.7170
455    67351.3825
456    67055.3975
Name: mavg, Length: 457, dtype: float64



#Golden ve Dead Cross ile al sat (SMA50-SMA200)

In [58]:
# Verileri al
end = datetime.now().strftime('%Y-%m-%d')
start = datetime.now() - pd.DateOffset(days=30)
data = yf.download("BTC-USD", start=start, end=end, interval="60m")

# Gerekli verileri seç
close = data['Close']
acilis_zamani = data.index
high = data['High']
low = data['Low']

def DCA():
    cuzdan = 100
    alimSayisi = 0
    satimSayisi = 0
    toplamCoin = 0
    komisyonOrani = 75/10000
    verilenKomisyon = 0

    # 50 ve 200 periyotlu basit hareketli ortalamaları hesapla
    sma50 = close.rolling(window=50).mean()
    sma200 = close.rolling(window=200).mean()

    print("############################")
    for i in range(len(close)):
        if not np.isnan(sma50[i]):
            if sma50[i-1] < sma200[i-1] and sma50[i] > sma200[i]:  # yukarı kesim
                print(zamanHesapla(acilis_zamani[i+1])," tarihinde ", cuzdan/close[i], " adet BTC alindi")
                alimSayisi += 1
                toplamCoin = cuzdan/close[i]
                verilenKomisyon += komisyonOrani*cuzdan

            # aşağı kesim
            if sma50[i-1] > sma200[i-1] and sma50[i] < sma200[i] and alimSayisi > 0:
                print(zamanHesapla(acilis_zamani[i+1])," tarihinde ", toplamCoin, " adet BTC satildi")
                satimSayisi += 1
                fiyat = close[i] * toplamCoin
                cuzdan = fiyat
                toplamCoin = 0
                verilenKomisyon += komisyonOrani*fiyat
                print(f"Bu iki işlem sonucunda işlem ederi : {cuzdan}")
                print("#####################################")

    print(f"Toplam yapılan islem : {(alimSayisi+satimSayisi)/2}")
    print(f"Toplam verilen komisyon : {verilenKomisyon}")
    print(f"Total cuzdan : {cuzdan}")

DCA()


[*********************100%%**********************]  1 of 1 completed

############################
2024-04-07 11:00:00  tarihinde  0.0014419763367176969  adet BTC alindi
2024-04-13 10:00:00  tarihinde  0.0014419763367176969  adet BTC satildi
Bu iki işlem sonucunda işlem ederi : 97.20365865446034
#####################################
2024-04-21 02:00:00  tarihinde  0.0014958474960196121  adet BTC alindi
2024-04-26 07:00:00  tarihinde  0.0014958474960196121  adet BTC satildi
Bu iki işlem sonucunda işlem ederi : 96.36701245184288
#####################################
Toplam yapılan islem : 2.0
Toplam verilen komisyon : 2.9308074732057268
Total cuzdan : 96.36701245184288


# Hareketli ortlamaya göre en iyi EMAD ve en kazançlı kriptoyu bulma

In [72]:
import pandas as pd

def ema(data, period):
    return data.ewm(span=period, adjust=False).mean()

def DCA(sembol, kisa_periyot, uzun_periyot):
    islemBasiDolar = 50
    alimSayisi = 0
    toplamBTC = 0
    komisyonOrani = 75 / 10000
    komisyon = 0

    csvName = sembol + '.csv'
    attributes = ["Open_time", 'Open', 'High', 'Low', 'Close', 'Volume', '1', '2', '3', '4', '5', '6']
    df = pd.read_csv(csvName, names=attributes)
    kapanis = df['Close']

    kisa_ema = ema(df['Close'], period=kisa_periyot)
    uzun_ema = ema(df['Close'], period=uzun_periyot)

    for i in range(df.shape[0]):
        if not pd.isna(kisa_ema[i]) and len(df) > 100 and i > 0:
            if uzun_ema[i - 1] < kisa_ema[i - 1] and uzun_ema[i] > kisa_ema[i]:  # yukarı kesim
                alimSayisi += 1
                toplamBTC += islemBasiDolar / kapanis[i]
                komisyon += komisyonOrani * islemBasiDolar

    '''print(f"Toplam yapılan işlem: {alimSayisi}")
    print(f"Toplam alınan {sembol}: {toplamBTC}")
    print(f"Toplam yatırım: {islemBasiDolar * alimSayisi}")
    print(f"Şuanki cüzdan tutarı: {toplamBTC * kapanis[len(kapanis) - 1]}")'''

    try:
        kat = (round(toplamBTC, 4) * kapanis[len(kapanis) - 1] - komisyon) / (islemBasiDolar * alimSayisi)
    except ZeroDivisionError:
        print(f"{sembol}'de hata var")
        return

    kazanc = toplamBTC * kapanis[len(kapanis) - 1] - (islemBasiDolar * alimSayisi) - komisyon

    '''print("Kaça katladı: ", kat)
    print("Toplam kazanc: ", kazanc)
    print("Toplam verilen komisyon: ", komisyon)
    print("---------------------------------------------------------------------------------")'''
    return [kat, kisa_periyot, uzun_periyot]

DCA("BTCUSDT", 2, 50)


[2.2096061218181817, 2, 50]

#Kod optimizasyonu (En iyi keşim noktaları, En çok karli crypto , En iyi EMA)

In [73]:
def optimize(sembolList):
  oranlar=[]
  dcalar = []
  totaloranlar=[]
  #emaList=[]
  uzunEmalar=[]
  kisaEmalar=[]
  ema_kesimler = []

  for i in sembolList:
    for x in range(2, 21):
      for y in range(50, 61):
        dcaFun = DCA(i, x, y)
        oranlar.append(dcaFun[0])
        uzunEmalar.append(dcaFun[1])
        kisaEmalar.append(dcaFun[2])
        ema_kesimler.append((dcaFun[1], dcaFun[2] , format(dcaFun[0], ".3f")))

    totaloranlar.append(max(oranlar))
    print("################################")
    print (f"coin adı : {i}")
    print(f"En iyi kazanç : {max(oranlar)}")
    print(f"En iyi EMA iki kesişim : {uzunEmalar[oranlar.index(max(oranlar))]} , {kisaEmalar[oranlar.index(max(oranlar))]}")

    #print(f"en_iyi_kazanc EMA : {oranlar.index(max(oranlar))+2}" )
    #emaList.append(oranlar.index(max(oranlar))+2)
    print("################################")
    oranlar.clear()

  print(f"Maximum kazanç : {max(totaloranlar)}")
  enCokKazandıranCoin = sembolList[totaloranlar.index(max(totaloranlar))]
  print(f"Maximum kazancı veren coin : {enCokKazandıranCoin}")
  kesisim_kaydet(ema_kesimler)


import pandas as pd

def kesisim_kaydet(ema_kesimler):
    df = pd.DataFrame(ema_kesimler, columns=['EMA1', 'EMA2' ,'EMA3'])
    df.to_csv('ema_kesimler.csv', index=False)
    print("EMA kesişim verileri başarıyla 'ema_kesimler.csv' dosyasına kaydedildi.")

optimize(['BTCUSDT'])

################################
coin adı : BTCUSDT
En iyi kazanç : 2.6313592933333334
En iyi EMA iki kesişim : 13 , 59
################################
Maximum kazanç : 2.6313592933333334
Maximum kazancı veren coin : BTCUSDT
EMA kesişim verileri başarıyla 'ema_kesimler.csv' dosyasına kaydedildi.


# Kesişim verilerin okunması

In [75]:
veri = pd.read_csv('ema_kesimler.csv')
print(veri)

     EMA1  EMA2   EMA3
0       2    50  2.210
1       2    51  2.210
2       2    52  2.202
3       2    53  2.202
4       2    54  2.186
..    ...   ...    ...
204    20    56  2.578
205    20    57  2.578
206    20    58  2.578
207    20    59  2.578
208    20    60  2.578

[209 rows x 3 columns]


# Gelen veri adedince düğüm oluşturulması ve javascripti html ye gömme

In [ ]:
dugumler = []
baglantilar = []
for i, row in veri.iterrows():
    # Her EMA sütunu için işlem yap
    for j, ema_sutunu in enumerate(['EMA1', 'EMA2', 'EMA3']):
        if j == 0 or j == 1:
            radius = min(20, row[ema_sutunu])
        else:
            radius = min(20, (row[ema_sutunu] - row['EMA1']) + (row['EMA2'] - row[ema_sutunu]))
        radius = round(radius, 2)

        # Düğüm verisini oluştur ve listeye ekle
        dugum = {
            'id': f'Düğüm {i * 3 + j + 1}',
            'group': 'Group',
            'radius': radius,
            'citing_patents_count': row[ema_sutunu]
        }
        dugumler.append(dugum)

        # İlk düğüm dışındaki düğümler için bağlantı oluştur
        if j > 0:
            fark = abs(row[ema_sutunu] - row[veri.columns[j - 1]])
            baglanti = {
                'source': f'Düğüm {i * 3 + j}',
                'target': f'Düğüm {i * 3 + j + 1}',
                'value': round(fark / 10, 4)
            }
            baglantilar.append(baglanti)

with open("veri.json", "w") as veri_file:
    json.dump({'nodes': dugumler, 'links': baglantilar}, veri_file)

subprocess.run(["python", "-m", "http.server"])


# **html sayfası**



```
<!-- index.html -->
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>D3.js Graph</title>
  <script src="https://d3js.org/d3.v7.min.js"></script>
  <style>
    svg {
      max-width: 100%;
      height: 100%;
      background-color: black;
    }
    text {
      fill: white;
    }
  </style>
</head>
<body>
  <script>
    const data = {
      nodes: {{ nodes | safe }},
      links: {{ links | safe }}
    };

    async function loadData() {
      const width = window.innerWidth;
      const height = window.innerHeight;
      const color = d3.scaleOrdinal(d3.schemeCategory10);
      const links = data.links.map(function(d) { return Object.assign({}, d); });
      const nodes = data.nodes.map(function(d) { return Object.assign({}, d); });

      const simulation = d3.forceSimulation(nodes)
        .force("link", d3.forceLink(links).id(function(d) { return d.id; }))
        .force("charge", d3.forceManyBody())
        .force("x", d3.forceX())
        .force("y", d3.forceY());

      const svg = d3.create("svg")
        .attr("width", width)
        .attr("height", height)
        .attr("viewBox", [-width / 2, -height / 2, width, height]);

      const link = svg.append("g")
        .attr("stroke", "#f00")
        .attr("stroke-opacity", 0.6)
        .selectAll("line")
        .data(links)
        .join("line")
        .attr("stroke-width", function(d) { return Math.sqrt(d.value); });

      const node = svg.append("g")
        .attr("stroke", "#fff")
        .attr("stroke-width", 1.5)
        .selectAll("g")
        .data(nodes)
        .join("g")
        .attr("transform", d => `translate(${d.x},${d.y})`);

      node.append("circle")
        .attr("r", d => d.radius * 1)
        .attr("fill", "black");

      node.append("text")
        .attr("text-anchor", "middle")
        .attr("dominant-baseline", "middle")
        .attr("font-size", "10px")
        .attr("fill", d => d.group === 'Group' ? 'red' : 'white')
        .text(d => d.citing_patents_count);

      node.call(d3.drag()
        .on("start", dragstarted)
        .on("drag", dragged)
        .on("end", dragended));

      simulation.on("tick", function() {
        link
          .attr("x1", function(d) { return d.source.x; })
          .attr("y1", function(d) { return d.source.y; })
          .attr("x2", function(d) { return d.target.x; })
          .attr("y2", function(d) { return d.target.y; });

        node
          .attr("transform", function(d) { return `translate(${d.x},${d.y})`; });
      });

      function dragstarted(event) {
        if (!event.active) simulation.alphaTarget(0.3).restart();
        event.subject.fx = event.subject.x;
        event.subject.fy = event.subject.y;
      }

      function dragged(event) {
        event.subject.fx = event.x;
        event.subject.fy = event.y;
      }

      function dragended(event) {
        if (!event.active) simulation.alphaTarget(0);
        event.subject.fx = null;
        event.subject.fy = null;
      }

      document.body.appendChild(svg.node());
    }

    window.onload = loadData;
  </script>
</body>
</html>
```



In [77]:
def zamanHesapla(timestamp):
    return datetime.fromtimestamp(timestamp / 1000) # Milisaniyeyi saniyeye dönüştür

def DCA(sembol, short_period, long_period, signal_period):
    islemBasiDolar = 50
    alimSayisi = 0
    toplamBTC = 0
    komisyonOrani = 75/10000
    komisyon = 0

    csvAdi = sembol + '.csv'
    sutunlar = ["Zaman",'Açılış','En Yüksek','En Düşük','Kapanış','Hacim','1','2','3','4','5','6']
    df = pd.read_csv(csvAdi, names=sutunlar)
    kapanis = df['Kapanış']

    # MACD çizgisini hesapla
    macd_gostergesi = ta.trend.macd(df['Kapanış'], window_fast=short_period, window_slow=long_period)
    df['macd_cizgisi'] = macd_gostergesi

    # Sinyal çizgisini hesapla
    sinyal_gostergesi = ta.trend.macd_signal(df['Kapanış'], window_fast=short_period, window_slow=long_period, window_sign=signal_period)
    df['sinyal_cizgisi'] = sinyal_gostergesi

    # MACD kesişimlerini bul
    crossings = np.where((df['macd_cizgisi'].shift(1) < df['sinyal_cizgisi'].shift(1)) & (df['macd_cizgisi'] > df['sinyal_cizgisi']), 1, 0)

    for i in range(df.shape[0]):
        if crossings[i] == 1 and len(df) > 100:
            alimSayisi += 1
            toplamBTC += islemBasiDolar / kapanis[i]
            komisyon += komisyonOrani * islemBasiDolar
            # Alım/satım değerlerini yazdır
            print(f"Alım/Satım değeri: {df['Kapanış'].iloc[i]}")

            # Alım/satım zamanını ve fiyatını yazdır
            print("Alım/Satım zamanı:", zamanHesapla(df['Zaman'].iloc[i]))
            print("Alım/Satım fiyatı:", df['Kapanış'].iloc[i])

            # Kaç kat kazanç elde edildiğini ve toplam kazancı hesapla
            try:
                kat = (np.round(toplamBTC, 4) * kapanis[len(kapanis) - 1] - komisyon) / (islemBasiDolar * alimSayisi)
            except:
                print(f"{sembol}'de hata var")

            kazanc = toplamBTC * kapanis[len(kapanis) - 1] - (islemBasiDolar * alimSayisi) - komisyon

            print("Kaça katladı: ", kat)
            print("Toplam kazanc: ", kazanc)
            print("Toplam verilen komisyon: ", komisyon)
            print("---------------------------------------------------------------------------------")
            return [kat, short_period, long_period, signal_period]

def optimize(sembolListesi):
    for sembol in sembolListesi:
        for short_period in range(2, 21):
            for long_period in range(50, 61):
                for signal_period in range(2, 10):
                    DCA(sembol, short_period, long_period, signal_period)

optimize(['BTCUSDT'])


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Toplam verilen komisyon:  0.375
---------------------------------------------------------------------------------
Alım/Satım değeri: 30196.78
Alım/Satım zamanı: 2023-04-11 00:00:00
Alım/Satım fiyatı: 30196.78
Kaça katladı:  2.1631745799999997
Toplam kazanc:  55.3372150110045
Toplam verilen komisyon:  0.375
---------------------------------------------------------------------------------
Alım/Satım değeri: 30196.78
Alım/Satım zamanı: 2023-04-11 00:00:00
Alım/Satım fiyatı: 30196.78
Kaça katladı:  2.1631745799999997
Toplam kazanc:  55.3372150110045
Toplam verilen komisyon:  0.375
---------------------------------------------------------------------------------
Alım/Satım değeri: 28354.39
Alım/Satım zamanı: 2023-03-29 00:00:00
Alım/Satım fiyatı: 28354.39
Kaça katladı:  2.29086132
Toplam kazanc:  62.20610296148146
Toplam verilen komisyon:  0.375
---------------------------------------------------------------------------------
Alım/Satım değeri: